In [1]:
from typing import List

from autogen_core import AgentId
from autogen_core import default_subscription
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core.tools import FunctionTool, Tool
from typing_extensions import Annotated

from agents.base_agent import BaseAgent
from context.cosmos_memory import CosmosBufferedChatCompletionContext
from helpers.fmputils import *
from helpers.yfutils import *
from datetime import date, timedelta, datetime
from helpers.summarizeutils import summarize, summarizeTopic
from helpers.analyzer import *
from helpers.reports import ReportLabUtils
from helpers.charting import ReportChartUtils
from azure.identity import ClientSecretCredential, DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, ContentSettings, generate_blob_sas
from config import Config
import uuid
from helpers.azureblob import azureBlobApi

In [5]:
async def copyReport(tenantId, clientId, clientSecret, blobAccountName, downloadPath, blobName, openAiBlobContainer):
    try:
        with open(downloadPath, "rb") as file:
            readBytes = file.read()
        credentials = ClientSecretCredential(tenantId, clientId, clientSecret)
        blobService = BlobServiceClient(
                "https://{}.blob.core.windows.net".format(blobAccountName), credential=credentials)
        blobClient = blobService.get_blob_client(container=openAiBlobContainer, blob=blobName)
        blobClient.upload_blob(readBytes,overwrite=True)
        return blobClient.url
    except Exception as e:
        print("Error in copyReport: ", e)
        return None

In [7]:
ticker_symbol = "MSFT"

blobFileName = "{}_{}Equity_Research_report.pdf".format(str(uuid.uuid4()), ticker_symbol)
localFileName = "reports\\{}_Equity_Research_report.pdf".format(ticker_symbol)
print("Local File Name: ", localFileName)
print("Blob File Name: ", blobFileName)

blobUrl = azureBlobApi.copyReport("reports\\{}_Equity_Research_report.pdf".format(ticker_symbol), blobFileName)

print("Blob URL: ", blobUrl)


Local File Name:  reports\MSFT_Equity_Research_report.pdf
Blob File Name:  b8990f6a-29e3-40c9-8ccf-98b1613ee2e4_MSFTEquity_Research_report.pdf
Blob api key found successfully.
Blob URL:  https://astdnapubstor.blob.core.windows.net/reports/b8990f6a-29e3-40c9-8ccf-98b1613ee2e4_MSFTEquity_Research_report.pdf


In [2]:
if Config.APP_IN_CONTAINER:
    reportDir = "/app/backend/reports/"
else:
    reportDir = "reports\\"

print("****************")
print("reportDir: ", reportDir)
print("****************")

****************
reportDir:  /app/backend/reports/
****************
